In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install transformers torch --quiet

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

model_name = "google/flan-t5-small"
model_dir = "/content/drive/MyDrive/Colab Notebooks/AI_CW_010/GenAI_model"

# Make sure the directory exists
os.makedirs(model_dir, exist_ok=True)

# Download from Hugging Face
print("⬇️ Downloading model...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Save to Drive
print("💾 Saving to Google Drive...")
tokenizer.save_pretrained(model_dir)
model.save_pretrained(model_dir)

print("✅ Done! Model saved at:", model_dir)


⬇️ Downloading model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

💾 Saving to Google Drive...
✅ Done! Model saved at: /content/drive/MyDrive/Colab Notebooks/AI_CW_010/GenAI_model


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Path
model_dir = "/content/drive/MyDrive/Colab Notebooks/AI_CW_010/GenAI_model"

# Load tokenizer and model from Drive
print("Loading model from Google Drive...")
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

print("✅ Model loaded successfully from Drive")


Loading model from Google Drive...
✅ Model loaded successfully from Drive


In [36]:
import json
from difflib import get_close_matches
from transformers import pipeline

# Build pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Intents, categories, urgency
intents = [
    "activate_my_card", "age_limit", "apple_pay_or_google_pay", "atm_support",
    "automatic_top_up", "balance_not_updated_after_bank_transfer",
    "balance_not_updated_after_cheque_or_cash_deposit", "beneficiary_not_allowed",
    "cancel_transfer", "card_about_to_expire", "card_acceptance", "card_arrival",
    "card_delivery_estimate", "card_linking", "card_not_working", "card_payment_fee_charged",
    "card_payment_not_recognised", "card_payment_wrong_exchange_rate", "card_swallowed",
    "cash_withdrawal_charge", "cash_withdrawal_not_recognised", "change_pin", "compromised_card",
    "contactless_not_working", "country_support", "declined_card_payment", "declined_cash_withdrawal",
    "declined_transfer", "direct_debit_payment_not_recognised", "disposable_card_limits",
    "edit_personal_details", "exchange_charge", "exchange_rate", "exchange_via_app",
    "extra_charge_on_statement", "failed_transfer", "fiat_currency_support",
    "get_disposable_virtual_card", "get_physical_card", "getting_spare_card",
    "getting_virtual_card", "lost_or_stolen_card", "lost_or_stolen_phone",
    "order_physical_card", "passcode_forgotten", "pending_card_payment",
    "pending_cash_withdrawal", "pending_top_up", "pending_transfer", "pin_blocked",
    "receiving_money", "Refund_not_showing_up", "request_refund",
    "reverted_card_payment?", "supported_cards_and_currencies", "terminate_account",
    "top_up_by_bank_transfer_charge", "top_up_by_card_charge",
    "top_up_by_cash_or_cheque", "top_up_failed", "top_up_limits", "top_up_reverted",
    "topping_up_by_card", "transaction_charged_twice", "transfer_fee_charged",
    "transfer_into_account", "transfer_not_received_by_recipient", "transfer_timing",
    "unable_to_verify_identity", "verify_my_identity", "verify_source_of_funds",
    "verify_top_up", "virtual_card_not_working", "visa_or_mastercard",
    "why_verify_identity", "wrong_amount_of_cash_received",
    "wrong_exchange_rate_for_cash_withdrawal"
]

categories = ["Card Services", "Transfer & Payments", "Account Management", "General Support"]
urgency_levels = ["Critical", "Medium", "Low"]

def classify_query(query: str):
    """
    Classify a query into intent, category, and urgency.
    Uses short prompt to avoid token limit issues.
    Always returns three values.
    """
    prompt = f"""
    Classify the following user query into intent, category, and urgency.
    Respond in JSON format with keys: intent, category, urgency.

    Query: "{query}"
    """

    output = pipe(prompt, max_new_tokens=100, clean_up_tokenization_spaces=True)
    prediction_text = output[0]["generated_text"].strip()

    # Try to parse JSON
    try:
        prediction_json = json.loads(prediction_text)
        predicted_intent = prediction_json.get("intent", prediction_text)
        predicted_category = prediction_json.get("category", "General Support")
        predicted_urgency = prediction_json.get("urgency", "Medium")
    except:
        # Fallback if JSON parsing fails
        predicted_intent = prediction_text
        predicted_category = "General Support"
        predicted_urgency = "Medium"

    # Map predicted intent to closest allowed intent
    best_intent = get_close_matches(predicted_intent, intents, n=1, cutoff=0.6)
    intent = best_intent[0] if best_intent else "Unknown"

    # Map category to closest allowed
    best_category = get_close_matches(predicted_category, categories, n=1, cutoff=0.6)
    category = best_category[0] if best_category else "General Support"

    # Map urgency to closest allowed
    best_urgency = get_close_matches(predicted_urgency, urgency_levels, n=1, cutoff=0.6)
    urgency = best_urgency[0] if best_urgency else "Medium"

    return intent, category, urgency


Device set to use cpu


In [37]:
# Try different customer queries
queries = [
    "I lost my card and need a replacement",
    "Why is my transfer taking so long?",
    "How can I change my passcode?",
    "My card got swallowed by the ATM",
    "Can I use Google Pay?"
]

for q in queries:
    intent, category, urgency = classify_query(q)
    print(f"Query: {q}")
    print(f" → Intent: {intent}")
    print(f" → Category: {category}")
    print(f" → Urgency: {urgency}")
    print("-" * 50)


Query: I lost my card and need a replacement
 → Intent: Unknown
 → Category: General Support
 → Urgency: Medium
--------------------------------------------------
Query: Why is my transfer taking so long?
 → Intent: Unknown
 → Category: General Support
 → Urgency: Medium
--------------------------------------------------
Query: How can I change my passcode?
 → Intent: Unknown
 → Category: General Support
 → Urgency: Medium
--------------------------------------------------
Query: My card got swallowed by the ATM
 → Intent: Unknown
 → Category: General Support
 → Urgency: Medium
--------------------------------------------------
Query: Can I use Google Pay?
 → Intent: Unknown
 → Category: General Support
 → Urgency: Medium
--------------------------------------------------


In [29]:
#Enter query and get the output...........
while True:
    user_query = input("Enter your query (or type 'exit' to stop): ")
    if user_query.lower() == "exit":
        break
    intent, category, urgency = classify_query(user_query)
    print(f" -Intent: {intent}")
    print(f" -Category: {category}")
    print(f" -Urgency: {urgency}")
    print("-" * 50)


Enter your query (or type 'exit' to stop):  I ordered a card but it has not arrived. Help please!
 -Intent: Unknown
 -Category: General Support
 -Urgency: Medium
--------------------------------------------------
Enter your query (or type 'exit' to stop): why my transfer taking so long?
 -Intent: Unknown
 -Category: General Support
 -Urgency: Medium
--------------------------------------------------
Enter your query (or type 'exit' to stop): can i know the status of my card delivery?
 -Intent: Unknown
 -Category: General Support
 -Urgency: Medium
--------------------------------------------------
Enter your query (or type 'exit' to stop): exit
